# imports

In [ ]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
# print("In module products sys.path[0], __package__ ==", sys.path[0], __package__)

# import os
import os.path
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from typing import Callable
import re
from math import floor
import statistics
from result_collection import result_collection

from test_runner import *
from test_runner.translators import *
from test_runner.analysers import SearchResult, QueryResultStatus


# from test_runner import TestCase, BaseTestRunner, LiftedPlanningRunner, GroundedPlanningRunner
# from test_runner.tapaal_caller import QueryResult

from parse_results import translator_result_type, search_result_type#, load_translator_results, load_search_results, 


# Style Config

In [ ]:
# downward = circle red
# grounded = square green
# lifted = triangle blue

# Load Data

In [ ]:
from result_collection import result_collection, result_container


infinite = float("inf")
results_dir = "../results_c2p"
plot_save_dir = "../results_c2p/plots"
os.makedirs(plot_save_dir, exist_ok=True)
results_path = "../results_c2p"

domain_sub_range = range(1, 32)


results = result_container()


def load_translator_results() -> translator_result_type:
    with open(os.path.join(results_path, f"translator_results.pickle"), "rb") as f:
        return pickle.load(f)


def load_search_results() -> search_result_type:
    with open(os.path.join(results_path, f"search_results.pickle"), "rb") as f:
        return pickle.load(f)



_loaded_translator_results: translator_result_type = load_translator_results()
_loaded_search_results: search_result_type = load_search_results()


# Translator -> domain -> result[]
translator_results_str_lookup: dict[str, dict[str, "BaseTranslator"]] = dict()


# Translator Results
for translator, domains in _loaded_translator_results.items():
    translator_results_str_lookup[translator.name] = dict()
    for domain, test_results in domains.items():
        translator_results_str_lookup[translator.name][domain.name] = test_results


# Search Results
domain_regex = re.compile(r"^(?P<domain>.+?) - .+?(?P<id>\d\d)$", re.MULTILINE)
for translator, translator_results in _loaded_search_results.items():
    for test_case, test_results in translator_results.items():
        for searcher, search_results in test_results.items():

            domain_name = domain_regex.match(test_case.name)["domain"]
            domain_sub_id = int(domain_regex.match(test_case.name)["id"])

            specific_translator_results = translator_results_str_lookup[translator.name][test_case.name]
            specific_searcher_results = search_results


            if (
                translator.name in results.translators and
                domain_name     in results.translators[translator.name].domains and
                searcher.name   in results.translators[translator.name].domains[domain_name].searchers
            ):
                results.translators[translator.name].domains[domain_name].searchers[searcher.name].set(domain_sub_id, specific_translator_results, specific_searcher_results)
            
            else:

                _result_collection = result_collection(domain_sub_range)
                _result_collection.set(domain_sub_id, specific_translator_results, specific_searcher_results)

                # results.set(translator.name, domain.name, searcher.name, _result_collection)
                results.set(translator.name, domain_name, searcher.name, _result_collection)




In [ ]:
for translator_name, translator_results in results.translators.items():
    for domain_name, test_results in translator_results.domains.items():
        for searcher_name, result_collection in test_results.searchers.items():

            if searcher_name == "no_color_optimizations":
                result_collection.style = None
                continue

            if translator_name == "colored" and searcher_name == "rpfs":
                result_collection.display_name = "color_rpfs"
                result_collection.style = "s-"
                result_collection.color="#332288"
                continue
            
            if translator_name == "colored" and searcher_name == "rpfs_safe_reductions":
                result_collection.display_name = "colored"
                result_collection.style = "s-"
                result_collection.color="#117733"
                continue         
               
            if translator_name == "colored" and searcher_name == "rpfs_safe_reductions_unlimited_intervals":
                result_collection.display_name = "colored_NT"
                result_collection.style = "s-"
                result_collection.color="#44AA99"
                continue

            if translator_name == "colored_hierarchy" and searcher_name == "rpfs":
                result_collection.display_name = "hier_rpfs"
                result_collection.style = "s-"
                result_collection.color="#88CCEE"
                continue

            if translator_name == "colored_hierarchy" and searcher_name == "rpfs_safe_reductions":
                result_collection.display_name = "hier_v1"
                result_collection.style = "s-"
                result_collection.color="#DDCC77"
                continue

            if translator_name == "colored_hierarchy" and searcher_name == "rpfs_safe_reductions_unlimited_intervals":
                result_collection.display_name = "hier_v1_NT"
                result_collection.style = "s-"
                result_collection.color="#CC6677"
                continue

            if translator_name == "colored_hierarchyv2" and searcher_name == "rpfs_safe_reductions":
                result_collection.display_name = "hier_v2"
                result_collection.style = "s-"
                result_collection.color="#ADD8E6"
                continue

            if translator_name == "colored_hierarchyv2" and searcher_name == "rpfs_safe_reductions_unlimited_intervals":
                result_collection.display_name = "hier_v2_NT"
                result_collection.style = "s-"
                result_collection.color="#00008B"
                continue

            if translator_name == "grounded" and searcher_name == "rpfs":
                result_collection.display_name = "grounded"
                result_collection.style = "v-"
                result_collection.color="#AA4499"
                continue

            if translator_name == "downward" and searcher_name == "lama_first":
                result_collection.display_name = "downward"
                result_collection.style = "o-"
                result_collection.color="#882255"
                continue

            if translator_name == "nothing" and searcher_name == "Tapaal":
                result_collection.display_name = "tapaal"
                result_collection.style = "s-"
                result_collection.color="#332288"
                continue
            
            if translator_name == "cpn" and searcher_name == "CpnToEnhsp":
                result_collection.display_name = "enhsp"
                result_collection.style = "o-"
                result_collection.color="#882255"
                continue

            print("Unstyled config")






# Plots

## Compare Success

In [ ]:


totalAgreedSolvable = 0
tapaalFasterSearchSolvable = 0
tapaalFasterTotalSolvable = 0
enhspFasterSearchSolvable = 0
enhspFasterTotalSolvable = 0
totalAgreedUnsolvable = 0

tapaalFasterSearchUnsolvable = 0
tapaalFasterTotalUnsolvable = 0
enhspFasterSearchUnsolvable = 0
enhspFasterTotalUnsolvable = 0

possiblyInvalidSolutionsInAgreed = 0
possiblyInvalidSolutionsTotal = 0

solvedTotalTapaal = 0
solvedTotalEnhsp = 0
unsolvedTotalTapaal = 0
unsolvedTotalEnhsp = 0

tapaal_undecided = 0
enhsp_undecided = 0
tapaal_error = 0
enhsp_error = 0


satisfiable_uniquely_solved_tapaal = 0
satisfiable_uniquely_solved_enhsp = 0
unsolvable_uniquely_solved_tapaal = 0
unsolvable_uniquely_solved_enhsp = 0


satisfiable_uniquely_solved_tapaal_other_timed_out = 0
satisfiable_uniquely_solved_enhsp_other_timed_out = 0
unsolvable_uniquely_solved_tapaal_other_timed_out = 0
unsolvable_uniquely_solved_enhsp_other_timed_out = 0

tapaal_timed_out = 0
enhsp_timed_out = 0


enhspErrors = dict()
enhspErrors["OutOfMemory"] = 0
enhspErrors["StackOverflow"] = 0
enhspErrors["SyntaxError"] = 0
enhspErrors["TranslationFailed"] = 0
enhspErrors["NullPointerException"] = 0


enhspErrors_translator = dict()
enhspErrors_translator["exprTooDeep_stringBuilder"] = 0
enhspErrors_translator["exprTooDeep_rawExpr"] = 0
enhspErrors_translator["success"] = 0
enhspErrors_translator["OutOfMemory"] = 0
enhspErrors_translator["StackOverflow"] = 0
enhspErrors_translator["NullPointerException"] = 0
enhspErrors_translator["PnmlParserError"] = 0


total_domains = 0
total_queries = 0

for domain_name, domain_results in results.domains.items():
    total_domains += 1

    success_states = dict()

    for translator_name, translator_results in domain_results.translators.items():
        for searcher_name, searcher_results in translator_results.searchers.items():

            if translator_name=="nothing" and searcher_name=="Tapaal":
                
                success_states["tapaal"] = [
                    (searcher_result[0].result_status,
                        translator_result[0].time.seconds_system+translator_result[0].time.seconds_user,
                        searcher_result[0].time.seconds_system+searcher_result[0].time.seconds_user,
                        translator_result[0],
                        searcher_result[0]
                        ) if searcher_result else None
                    for (translator_result, searcher_result) in zip(searcher_results.translator_results, searcher_results.search_results)
                ]

                continue

            if translator_name=="cpn" and searcher_name=="CpnToEnhsp":

                success_states["enhsp"] = [
                    (searcher_result[0].result_status,
                        translator_result[0].time.seconds_system+translator_result[0].time.seconds_user,
                        searcher_result[0].time.seconds_system+searcher_result[0].time.seconds_user,
                        translator_result[0],
                        searcher_result[0]
                        ) if searcher_result else None
                    for (translator_result, searcher_result) in zip(searcher_results.translator_results, searcher_results.search_results)
                ]



    tapaal_results = success_states["tapaal"]
    enhsp_results = success_states["enhsp"]

    #if tapaal_results != enhsp_results:
    disagreeCount = 0
    agreedSolvable = 0
    agreedUnsolvable = 0
    for i in range(0,31):
        if (not tapaal_results[i]) and (not enhsp_results[i]):
            continue

        total_queries += 1

        tapaal_result, tapaal_translate_time, tapaal_search_time, tapaal_translator_result, tapaal_search_result = tapaal_results[i] #if tapaal_results[i] else (None, infinite, infinite, None)
        enhsp_result, enhsp_translate_time, enhsp_search_time, enhsp_translator_result, enhsp_search_result = enhsp_results[i] #if enhsp_results[i] else (None, infinite, infinite, None)

        # Disagree on solvable or not
        if ( tapaal_result == QueryResultStatus.satisfied and enhsp_result == QueryResultStatus.unsolvable or
             tapaal_result == QueryResultStatus.unsolvable and enhsp_result == QueryResultStatus.satisfied):
             disagreeCount += 1

        if tapaal_result == QueryResultStatus.satisfied:
            solvedTotalTapaal += 1
        if enhsp_result == QueryResultStatus.satisfied and not enhsp_search_result.plan_warning:
            solvedTotalEnhsp += 1

        if tapaal_result == QueryResultStatus.unsolvable:
            unsolvedTotalTapaal += 1
        if enhsp_result == QueryResultStatus.unsolvable and not enhsp_search_result.plan_warning:
            unsolvedTotalEnhsp += 1

        if tapaal_result == QueryResultStatus.unknown and not tapaal_search_result.timed_out:
            tapaal_undecided += 1
        if enhsp_result == QueryResultStatus.unknown:
            enhsp_undecided += 1

        if tapaal_result == QueryResultStatus.error:
            tapaal_error += 1
        if enhsp_result == QueryResultStatus.error:
            enhsp_error += 1
            errorsAtOnce = 0
            if enhsp_search_result.get("OutOfMemory", False):
                enhspErrors["OutOfMemory"]  += 1
                errorsAtOnce += 1
            if enhsp_search_result.get("StackOverflow", False):
                enhspErrors["StackOverflow"]  += 1
                errorsAtOnce += 1
            if enhsp_search_result.get("SyntaxError", False): 
                enhspErrors["SyntaxError"]  += 1
                errorsAtOnce += 1
            if enhsp_search_result.get("TranslationFailed", False):
                enhspErrors["TranslationFailed"]  += 1
                errorsAtOnce += 1
            if enhsp_search_result.get("NullPointerException", False) and not enhsp_search_result.get("TranslationFailed", False):
                enhspErrors["NullPointerException"]  += 1
                errorsAtOnce += 1
            errorsAtOnce

            errorsAtOnce_translator = 0
            if enhsp_translator_result.get("exprTooDeep_stringBuilder", False):
                enhspErrors_translator["exprTooDeep_stringBuilder"] += 1
                errorsAtOnce_translator += 1
            if enhsp_translator_result.get("exprTooDeep_rawExpr", False):
                enhspErrors_translator["exprTooDeep_rawExpr"] += 1
                errorsAtOnce_translator += 1
            if enhsp_translator_result.get("success", False):
                enhspErrors_translator["success"] += 1
                errorsAtOnce_translator += 1
            if enhsp_translator_result.get("OutOfMemory", False):
                enhspErrors_translator["OutOfMemory"] += 1
                errorsAtOnce_translator += 1
            if enhsp_translator_result.get("StackOverflow", False) and not enhsp_translator_result.get("exprTooDeep_stringBuilder", False) and not enhsp_translator_result.get("exprTooDeep_rawExpr", False):
                enhspErrors_translator["StackOverflow"] += 1
                errorsAtOnce_translator += 1
            if enhsp_translator_result.get("NullPointerException", False):
                enhspErrors_translator["NullPointerException"] += 1
                errorsAtOnce_translator += 1
            if enhsp_translator_result.get("PnmlParserError", False):
                enhspErrors_translator["PnmlParserError"] += 1
                errorsAtOnce_translator += 1

            errorsAtOnce_translator

        if tapaal_result == QueryResultStatus.satisfied and not enhsp_result == QueryResultStatus.satisfied:
            satisfiable_uniquely_solved_tapaal += 1
            if enhsp_result == QueryResultStatus.timeout:
                satisfiable_uniquely_solved_tapaal_other_timed_out += 1

        if enhsp_result == QueryResultStatus.satisfied and not tapaal_result == QueryResultStatus.satisfied:
            satisfiable_uniquely_solved_enhsp += 1
            if tapaal_search_result and tapaal_search_result.timed_out:
                satisfiable_uniquely_solved_enhsp_other_timed_out += 1

            

        if tapaal_result == QueryResultStatus.unsolvable and not enhsp_result == QueryResultStatus.unsolvable:
            unsolvable_uniquely_solved_tapaal += 1
            if enhsp_result == QueryResultStatus.timeout:
                unsolvable_uniquely_solved_tapaal_other_timed_out += 1
        if enhsp_result == QueryResultStatus.unsolvable and not tapaal_result == QueryResultStatus.unsolvable:
            unsolvable_uniquely_solved_enhsp += 1
            if tapaal_search_result and tapaal_search_result.timed_out:
                unsolvable_uniquely_solved_enhsp_other_timed_out += 1



        if enhsp_search_result and enhsp_search_result.plan_warning:
            possiblyInvalidSolutionsTotal += 1
                                
        # Agree on solvable
        if tapaal_result == QueryResultStatus.satisfied and enhsp_result == QueryResultStatus.satisfied and not enhsp_search_result.plan_warning:
            totalAgreedSolvable += 1
            agreedSolvable +=1
            if enhsp_search_time < tapaal_search_time:
                enhspFasterSearchSolvable += 1
            else:
                tapaalFasterSearchSolvable += 1
            if enhsp_translate_time + enhsp_search_time < tapaal_translate_time + tapaal_search_time:
                enhspFasterTotalSolvable += 1
            else:
                tapaalFasterTotalSolvable += 1

            if enhsp_search_result and enhsp_search_result.plan_warning:
                possiblyInvalidSolutionsInAgreed += 1

        # Agree on unsolvable
        if tapaal_result == QueryResultStatus.unsolvable and enhsp_result == QueryResultStatus.unsolvable:
            totalAgreedUnsolvable += 1
            agreedUnsolvable += 1
            if enhsp_search_time < tapaal_search_time:
                 enhspFasterSearchUnsolvable += 1
            else:
                tapaalFasterSearchUnsolvable += 1

            if enhsp_translate_time + enhsp_search_time < tapaal_translate_time + tapaal_search_time:
                 enhspFasterTotalUnsolvable += 1
            else:
                tapaalFasterTotalUnsolvable += 1
        
        if tapaal_search_result and tapaal_search_result.timed_out:
            tapaal_timed_out += 1
        if enhsp_result == QueryResultStatus.timeout:
            enhsp_timed_out += 1
    
    # Skip boring domains
    if disagreeCount == 0 and agreedSolvable and agreedUnsolvable == 0:
        continue

    print(domain_name)
    print("\tSolvable:", agreedSolvable, "Unsolvable:", agreedUnsolvable, "disagreed on:", disagreeCount)
    if disagreeCount > 0:
        print("\ttapaal", success_states["tapaal"])
        print("\tenhsp", success_states["enhsp"])


print(f"ENHSP warnings in agreed: {possiblyInvalidSolutionsInAgreed}")
print(f"ENHSP warnings in total: {possiblyInvalidSolutionsTotal}")

print(f"Solved by Tapaal: {solvedTotalTapaal}")
print(f"Solved by Enhsp: {solvedTotalEnhsp}")
print(f"Total agreed solvable: {totalAgreedSolvable}")

print(f"Tapaal Faster Search solvable: {tapaalFasterSearchSolvable}")
print(f"Tapaal Faster Total solvable: {tapaalFasterTotalSolvable}")
print(f"ENHSP Faster Search solvable: {enhspFasterSearchSolvable}")
print(f"ENHSP Faster Total solvable: {enhspFasterTotalSolvable}")

print(f"Unsolved by Tapaal: {unsolvedTotalTapaal}")
print(f"Unsolved by Enhsp: {unsolvedTotalEnhsp}")
print(f"Total agreed unsolvable: {totalAgreedUnsolvable}")
print(f"Tapaal Faster Search unsolvable: {tapaalFasterSearchUnsolvable}")
print(f"Tapaal Faster Total unsolvable: {tapaalFasterTotalUnsolvable}")
print(f"ENHSP Faster Search unsolvable: {enhspFasterSearchUnsolvable}")
print(f"ENHSP Faster Total unsolvable: {enhspFasterTotalUnsolvable}")


print(f"satisfiable_uniquely_solved_tapaal: {satisfiable_uniquely_solved_tapaal}")
print(f"satisfiable_uniquely_solved_enhsp: {satisfiable_uniquely_solved_enhsp}")
print(f"unsolvable_uniquely_solved_tapaal: {unsolvable_uniquely_solved_tapaal}")
print(f"unsolvable_uniquely_solved_enhsp: {unsolvable_uniquely_solved_enhsp}")

print(f"satisfiable_uniquely_solved_tapaal_other_timed_out:", satisfiable_uniquely_solved_tapaal_other_timed_out)
print(f"satisfiable_uniquely_solved_enhsp_other_timed_out:", satisfiable_uniquely_solved_enhsp_other_timed_out)
print(f"unsolvable_uniquely_solved_tapaal_other_timed_out:", unsolvable_uniquely_solved_tapaal_other_timed_out)
print(f"unsolvable_uniquely_solved_enhsp_other_timed_out:", unsolvable_uniquely_solved_enhsp_other_timed_out)


print(f"tapaal_undecided: {tapaal_undecided}")
print(f"enhsp_undecided: {enhsp_undecided}")

print(f"tapaal_error: {tapaal_error}")
print(f"enhsp_error: {enhsp_error}")
print(f"enhspErrors_translator: {enhspErrors_translator}")
print(f"enhspErrors: ", enhspErrors)

print(f"tapaal_timed_out:", tapaal_timed_out)
print(f"enhsp_timed_out:", enhsp_timed_out)

print("Total Domains: ", total_domains)
print("Total Queries: ", total_queries)

print("table:", "tapaal", "enhsp")
print("Satisfiable Queries Answered:                   ", solvedTotalTapaal, f"{solvedTotalEnhsp} + {possiblyInvalidSolutionsTotal} flagged")
print("Satisfiable Uniquely Answered:                  ", satisfiable_uniquely_solved_tapaal, satisfiable_uniquely_solved_enhsp)
print("Satisfiable Uniquely Answered: (other timed out)", satisfiable_uniquely_solved_tapaal_other_timed_out, satisfiable_uniquely_solved_enhsp_other_timed_out)
print("Satisfiable Total Faster:                       ", tapaalFasterTotalSolvable, enhspFasterTotalSolvable)
print("Satisfiable Total Faster:                       ", tapaalFasterSearchSolvable, enhspFasterSearchSolvable)
print("---")
print("Unsolvable Queries Answered:                    ", unsolvedTotalTapaal, unsolvedTotalEnhsp)
print("Unsolvable Uniquely Answered:                   ", unsolvable_uniquely_solved_tapaal, unsolvable_uniquely_solved_enhsp)
print("Unsolvable Uniquely Answered: (other timed out) ", unsolvable_uniquely_solved_tapaal_other_timed_out, unsolvable_uniquely_solved_enhsp_other_timed_out)
print("Unsolvable Total Faster:                        ", tapaalFasterTotalUnsolvable, enhspFasterTotalUnsolvable)
print("Unsolvable Search Faster:                       ", tapaalFasterSearchUnsolvable, enhspFasterSearchUnsolvable)
print("---")
print("Timed Out:                                      ", tapaal_timed_out, enhsp_timed_out)
print("Error:                                          ", tapaal_error, enhsp_error)




## Count Reductions

In [ ]:
# import pandas as pd

reductions = dict()
domains_per_reduction = dict()


for translator_name, translator_results in results.translators.items():
    # if translator_name == "colored": continue
    # if translator_name == "colored_hierarchy": continue
    if translator_name == "grounded": continue
    if translator_name == "downward": continue

    for searcher_name, searcher_results in translator_results.searchers.items():
        for domain_name, result_collections in searcher_results.domains.items():

            for domain_sub_id, search_results in enumerate(result_collections.search_results):
                
                if len(result_collection.search_results) == 0:
                    continue

                search_result = result_collection.search_results[0]
                if hasattr(search_result, "reductions_applied"):
                    for (rule_id, count) in search_result.reductions_applied.items():
                        if not rule_id in reductions:
                            reductions[rule_id] = 0
                            domains_per_reduction[rule_id] = list()
                        
                        reductions[rule_id] += count
                        if count > 0:
                            domains_per_reduction[rule_id].append(f"{translator.name}:{domain_name.name}_{domain_sub_id}")
    
    #print(test_case.name)
for (reduction_id, count) in reductions.items():
    print(str.ljust(f"{reduction_id}: {count}", 30) + f"{domains_per_reduction[rule_id]}")


    

# -r 3 2,3,4,6,8,11,12

# Overview Plot
plot = run config

x = IPC instance id

y = median time

In [ ]:
from test_runner.analysers import TranslatorResult, SearchResult
from result_collection import result_collection

# Generic


def median_reduction(translator_reduction: Callable[[TranslatorResult], float], searcher_reduction: Callable[[SearchResult], float]) -> Callable[[result_collection], list[float]]:
    def reduction(rc: result_collection) -> list[float]:
        translator_vals_per_task = [[translator_reduction(x) for x in translator_results] for translator_results in rc.translator_results]
        search_vals_per_task     = [[searcher_reduction(x) for x in search_results] for search_results in rc.search_results]


        translator_numeric = [statistics.median(results) if len(results) > 0 else infinite for results in translator_vals_per_task]
        searcher_numeric   = [statistics.median(results) if len(results) > 0 else infinite for results in search_vals_per_task]


        added = [ x + y for x, y in zip(translator_numeric, searcher_numeric)]

        return added
    
    return reduction


def minimum_reduction(translator_reduction: Callable[[TranslatorResult], float], searcher_reduction: Callable[[SearchResult], float]) -> Callable[[result_collection], list[float]]:
    def reduction(rc: result_collection) -> list[float]:
        translator_vals_per_task = [[translator_reduction(x) for x in translator_results] for translator_results in rc.translator_results]
        search_vals_per_task     = [[searcher_reduction(x) for x in search_results] for search_results in rc.search_results]


        translator_numeric = [statistics.median(results) if len(results) > 0 else infinite for results in translator_vals_per_task]
        searcher_numeric   = [statistics.median(results) if len(results) > 0 else infinite for results in search_vals_per_task]


        added = [ x + y for x, y in zip(translator_numeric, searcher_numeric)]

        return added
    
    return reduction


def median_search_property(prop_name: str) -> Callable[[result_collection], list[float]]:
    return median_reduction(
        lambda res: 0,
        lambda res: res.get(prop_name, infinite)
    )


# Time

median_time_total = median_reduction(
    lambda res: res.time.seconds_system + res.time.seconds_user,
    lambda res: res.time.seconds_system + res.time.seconds_user if res.has_plan else infinite
)


minimum_time_search = minimum_reduction(
    lambda res: 0,
    lambda res: res.time.seconds_system + res.time.seconds_user if res.has_plan else infinite
)

median_time_translate = minimum_reduction(
    lambda res: res.time.seconds_system + res.time.seconds_user if res.valid_translation else infinite,
    lambda res: 0
)

median_time_search = median_reduction(
    lambda res: 0,
    lambda res: res.time.seconds_system + res.time.seconds_user if res.has_plan else infinite
)


minimum_time_total = minimum_reduction(
    lambda res: res.time.seconds_system + res.time.seconds_user,
    lambda res: res.time.seconds_system + res.time.seconds_user if res.has_plan else infinite
)





## Total time scatterplot

In [ ]:
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D

solved_median_time_total = median_reduction(
    lambda res: res.time.seconds_system + res.time.seconds_user,
    lambda res: res.time.seconds_system + res.time.seconds_user if (res.result_status == QueryResultStatus.satisfied  or res.result_status == QueryResultStatus.unsolvable) else infinite
)


x = []
y = []
styles = []

for domain_name, domain_results in results.domains.items():
    tapaal = domain_results.translators["nothing"].searchers["Tapaal"]
    enhsp = domain_results.translators["cpn"].searchers["CpnToEnhsp"]

    tapaal_time = solved_median_time_total(tapaal)
    enshp_time = solved_median_time_total(enhsp)

    for i in range(0, 31):
        # Experiment actually exists if Tapaal had a translation
        if not tapaal.translator_results[i]:
            continue

        if tapaal_time[i] == infinite and enshp_time[i] == infinite:
            continue # Would just be in top right

        status_provider = tapaal.search_results[i][0]
        if status_provider.result_status == QueryResultStatus.error:
            status_provider = enhsp.search_results[i][0]

        
        solvable = status_provider.result_status == QueryResultStatus.satisfied
        unsolvable = status_provider.result_status == QueryResultStatus.unsolvable
        #unknown = status_provider.result_status == QueryResultStatus.unknown
        error = status_provider.result_status == QueryResultStatus.error
        warning = enhsp.search_results[i][0].plan_warning

        style = (
            ("x", "#A00") if warning else
            ("^", "#0A0") if solvable else
            ("H", "#00A") if unsolvable else
            #("o", "#222") if unknown else
            ("s", "#FF0") if error else
                None
        )
        if not style:
            raise "Unhandled State"
        
        styles.append(style)

        x.append(tapaal_time[i])
        y.append(enshp_time[i])


plt.style.use('_mpl-gallery')




max_x = max([i for i in x if i != infinite])
max_y = max([i for i in y if i != infinite])
max_max = max(max_x, max_y)

unsolvedVal = 10**5 # max_max * 3
plot_max = max(max_max, unsolvedVal)


# fig = plt.figure(figsize=(scale, scale))
fig = plt.figure(figsize=(5, 4))
ax = plt.gca()

shapes, colors = zip(*styles)

ax.plot([0, plot_max], [0, plot_max])

for i in range(len(x)):
    plotX = x[i] if x[i] != infinite else unsolvedVal
    plotY = y[i] if y[i] != infinite else unsolvedVal

    ax.scatter(plotX, plotY, c=colors[i], marker=shapes[i])#, vmin=0, vmax=100)

plt.xscale("log")
plt.yscale("log")
plt.xlabel("Total seconds TAPAAL")
plt.ylabel("Total seconds ENHSP")

ax.set_xlim(xmin=0.001, xmax=plot_max)
ax.set_ylim(ymin=0.001, ymax=plot_max)



labels = [item.get_text() for item in ax.get_xticklabels()]
labels[-2] = 'Undecided'
ax.set_xticklabels(labels)

labels = [item.get_text() for item in ax.get_yticklabels()]
labels[-2] = 'Undecided'
ax.set_yticklabels(labels)


warning    = Line2D([0], [0], label='warning',      marker="x", c="#A00", markersize=10, markerfacecolor="#A00", linestyle='')
solvable   = Line2D([0], [0], label='Satisfiable',  marker="^", c="#0A0", markersize=10, markerfacecolor="#0A0", linestyle='')
unsolvable = Line2D([0], [0], label='Unsolvable',   marker="H", c="#00A", markersize=10, markerfacecolor="#00A", linestyle='')


plt.legend(handles=[warning, solvable, unsolvable], loc="lower right")

# plt.show()
plt.tight_layout()
plt.savefig(os.path.join(plot_save_dir, f"cpnToPddlResults_Total.png"))



# Old cactus plots

In [ ]:
# System + User Time

import matplotlib


# class XTickFormater(matplotlib.ticker.Formatter):
#     def format_ticks(values):
#         return [f"P{v+1}" for v in values]
# xtickFormatter = XTickFormater()
# Formerly 1+ to x axis
xtickFormatter = matplotlib.ticker.FuncFormatter(lambda x, pos: f"P{x:02.0f}")
xtickLocator = matplotlib.ticker.MaxNLocator(integer=True)


def make_overview_cactus_plot(name: str, description: str, reduction: Callable[["result_collection"], list[float]]):
    fig_rows = 5
    fig_cols = 10


    # Domain -> searcher -> translator -> IPCMedian[] 
    # search_results_domain_searcher_translator_test_case


    i=1
    # scale = 4.5
    scale = 10
    # fig = plt.figure(figsize=(2.3*scale, scale))
    fig = plt.figure(figsize=(scale, scale))
#    ax = fig.gca()
    # plt.title(description)

    for domain_name, domain_results in results.domains.items():


        subplt = plt.subplot(fig_rows, fig_cols, i)
        max_y = 0
        axis = plt.gca()

        for translator_name, translator_results in domain_results.translators.items():
            for searcher_name, _result_collection in translator_results.searchers.items():

                #plt.plot(list(median_times.values())[0], 'o-', label=f"{translator.name}({searcher.name})")
                # plt.plot(median_times, 'o-', label=f"{translator.name}({searcher.name})")
                reduced_data = reduction(_result_collection)
                if _result_collection.style != None and len([x for x in reduced_data if x != infinite]) > 0:
                    
                    plt.plot(reduced_data, _result_collection.style, color=_result_collection.color, label=_result_collection.display_name)#f"{translator.name}({searcher.name})")

                    #plt.xticks(range(0,30), [f"P{x+1}" for x in range(0,30)])
                    # plt.xlabels([f"P{x+1}" for x in range(0,30)])
                    #(xlim_min, xlim_max) = axis.get_xlim()
                    #axis.set_xlim([xlim_min, min(xlim_max, 30)])

                    # yaxis begins at 0
                    #(ylim_min, ylim_max) = axis.get_ylim()
                    max_y = max(max_y, max([x for x in reduced_data if x != infinite]))


                    subplt.xaxis.set_major_formatter(xtickFormatter)
                    subplt.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
                    # subplt.axes.ticklabel_format(xtickFormatter, "x")
                    # fig.gca().axes.xaxis.format_ticks(xtickFormatter)
                #yticks = range(0,30)
                #plt.yticks(yticks, step=1)
#                ax.xaxis.set_major_locator(MaxNLocator(integer=True)) # Integer x axis

        # axis.set_ylim(bottom=0)

        plt.title(domain_name)
        #plt.title(f'{description} - {test_case.name}')
        plt.gca().legend(loc='best')
        plt.xlabel('Task index')
        if (i-1)%3 == 0:
            plt.ylabel(description)
        subplt.set_yscale("log")
        i += 1


    plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
    plt.tight_layout()
    plt.savefig(os.path.join(plot_save_dir, f"{name}.png"))



make_overview_cactus_plot("overview_med_time_total", 'Median Time (sec)', median_time_total)
make_overview_cactus_plot("overview_min_time_total", "Min Total Time", minimum_time_total)
make_overview_cactus_plot("overview_min_time_search", "Min Search Time", minimum_time_search)



# #make_overview_cactus_plot("overview_min_time", "Min Total Time", lambda res: res.min_time)
# #make_overview_cactus_plot("overview_min_verificaiton_time", 'Min verification time', lambda resultCollection: min([res.get("time_verification", infinite) for res in resultCollection.raw])) 

# make_overview_cactus_plot("overview_time", 'Median Time (sec)', lambda res: res.median_time)
# #make_overview_cactus_plot("overview_time_minus_unfolding", 'Median Time (sec) - minus unfolding', lambda res: res.median_time_minus_unfolding)
# #make_overview_cactus_plot("overview_time_minus_unfolding", 'Median unfolding Time (sec)', lambda res: res.median_unfolding_time)
# make_overview_cactus_plot("overview_verification_time", "Median Verification Time", lambda res: res.median_verification_time)
# # make_overview_cactus_plot("overview_unfolded_places", "Unfolded Place Count", lambda resultCollection: resultCollection.raw[0].get("place_count_after_reduction", infinite))
# # make_overview_cactus_plot("overview_unfolded_transitions", "Unfolded Transition Count", lambda resultCollection: resultCollection.raw[0].get("transition_count_after_reduction", infinite))


# Single Domain Overview

In [ ]:
# System + User Time

import matplotlib

xtickFormatter = matplotlib.ticker.FuncFormatter(lambda x, pos: f"P{x:02.0f}")
xtickLocator = matplotlib.ticker.MaxNLocator(integer=True)


def make_domain_cactus_plot(name: str, description: str, rows, cols, reductions: list[tuple[str, Callable[["result_collection"], list[float]]]]):
    fig_rows = rows #1
    fig_cols = cols #len(reductions)

    scale = 10

#    ax = fig.gca()
    # plt.title(description)
    for (domain_name, domain_results) in results.domains.items():
        fig = plt.figure(figsize=(scale, scale))
        i=1
        for (reduction_name, reduction) in reductions:
            subplt = plt.subplot(fig_rows, fig_cols, i)
            max_y = 0
            axis = plt.gca()
            for (translator, translator_results) in domain_results.translators.items():
                for (searcher, resultCollections) in translator_results.searchers.items():
                    reduced_data = reduction(resultCollections)
                    if resultCollections.style != None and len([x for x in reduced_data if x != infinite]) > 0:

                        # if searcher.display_name != "color_rpfs_safe_no_timeout" and searcher.display_name != "hier_rpfs_safe_no_timeout" and searcher.display_name != "grounded":
                        #     continue
                        
                        plt.plot(reduced_data, resultCollections.style, color=resultCollections.color, label=resultCollections.display_name)

                        max_y = max(max_y, max([x for x in reduced_data if x != infinite]))


                        subplt.xaxis.set_major_formatter(xtickFormatter)
                        subplt.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))


            plt.title(reduction_name)
            #plt.title(f'{description} - {test_case.name}')
            plt.gca().legend(loc='best')
            plt.xlabel('Task index')
            if (i-1)%4 == 0:
                plt.ylabel(domain_name)
            subplt.set_yscale("log")
            i += 1


        plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
        plt.tight_layout()
        plt.savefig(os.path.join(plot_save_dir, f"{name}_{domain_name}.png"))

#make_overview_cactus_plot("overview_min_time", "Min Total Time", lambda res: res.min_time)
#make_overview_cactus_plot("overview_min_verificaiton_time", 'Min verification time', lambda resultCollection: min([res.get("time_verification", infinite) for res in resultCollection.raw])) 

#make_overview_cactus_plot("overview_time", 'Median Time (sec)', lambda res: res.median_time)
#make_overview_cactus_plot("overview_time_minus_unfolding", 'Median Time (sec) - minus unfolding', lambda res: res.median_time_minus_unfolding)
#make_overview_cactus_plot("overview_time_minus_unfolding", 'Median unfolding Time (sec)', lambda res: res.median_unfolding_time)

reductions:list[tuple[str, Callable[["result_collection"], float]]] = {
    ('Median Time (sec)', median_time_total),
    ("Median Search Time (sec)", median_time_search),
    ("Median Translate Time (sec)", median_time_translate),
    # ("Median Verification Time",  median_search_property("time_verification")),
    ("Unfolded Place Count", median_search_property("place_count_after_reduction")),
    ("Unfolded Transition Count", median_search_property("transition_count_after_reduction"))
}

make_domain_cactus_plot("name", "description", 3, 2, reductions)
# make_domain_cactus_plot("name", "description", 2, 2, reductions)

## Reductions

In [ ]:
# min_time              ", "Min Total Time                     ", lambda res: res.min_time)




# min_verificaiton_time ", 'Min verification time              ', lambda resultCollection: min([res.get("time_verification", infinite) for res in resultCollection.raw])) 




# time                  ", 'Median Time (sec)                  ', lambda res: res.median_time)



# time_minus_unfolding  ", 'Median Time (sec) - minus unfolding', lambda res: res.median_time_minus_unfolding)

# make_overview_cactus_plot("overview_time_minus_unfolding", 'Median Time (sec) - minus unfolding', lambda res: res.median_time_minus_unfolding)

def get_median_time_minus_unfolding(res: ResultCollection):
                
                
                if translator.name=="colored":
                
                plan_lengths = [len(result.plan.actions) if result.get("has_plan", False) else float("inf") for result in results ]
                resultCollection.median_trace_length = statistics.median(plan_lengths)

                time_unfolding = [result["time_unfolding"] if result.get("has_plan", False) else float("inf") for result in results ]
                resultCollection.median_unfolding_time = statistics.median(time_unfolding)

                resultCollection.median_time_minus_unfolding = statistics.median([(result.time.seconds_system + result.time.seconds_user - result["time_unfolding"]) if result.get("has_plan", False) else float("inf") for result in results ])



# time_minus_unfolding  ", 'Median unfolding Time (sec)        ', lambda res: res.median_unfolding_time)


# verification_time     ", "Median Verification Time           ", lambda res: res.median_verification_time)


# unfolded_places       ", "Unfolded Place Count               ", lambda resultCollection: resultCollection.raw[0].get("place_count_after_reduction", infinite))


# unfolded_transitions  ", "Unfolded Transition Count          ", lambda resultCollection: resultCollection.raw[0].get("transition_count_after_reduction", infinite))
